###### Information

> - 당연히 일어날 일에 대한 소식은 별 정보가 아니다. 반면 일어날 것 같지 않은 일에 대한 소식은 큰 정보라 할 수 있다. 즉 사건이 발생할 확률에 반비례하계 정보량을 정의할 수 있다.
$$I = - \log p(x)$$


> - 평균 정보량을 엔트로피(entropy)라 한다.
$$H = \mathbb{E}[I] =  - \sum p(x) \log p(x)$$

> - 분포 $p$를 기준으로한 **분포 $q$의 KL divergence**(KL div $q$ w.r.t $p$), $KL(p \Vert q)$를 정보량&엔트로피 관점에서 생각해볼 수 있다. 
 - 우선 분포 $p$와 분포 $q$의 엔트로피 자이는 아래와 같다.
 $$\begin{align}
 -\sum \color{red}{q(x)} \log q(x) + \sum p(x) \log p(x)
 \end{align}$$
 - 위 식에서 기댓값의 기준 확률을 $p$로 통일하면 아래와 같이 $KL(p \Vert q)$가 된다.
 $$\begin{align}
 & -\sum \color{red}{p(x)} \log q(x) + \sum p(x) \log p(x) \\
 & = p(x) \log \frac{p(x)}{q(x)} \\
 & = KL(p \Vert q)
 \end{align}$$

### Variational Inference
아래 페이지를 요약한다.
- https://ermongroup.github.io/cs228-notes/inference/variational/

> - 어떤 미지의 확률분포 $p$에 근접한 분포 $q$를 추론하는 문제가 있다고 할 때, 이를 풀 수 있는 방법들중 하나가 Variational inference이다. 결론부터 말하자면 분포에 대한 추론(inference) 문제를 최적화(optimization) 문제로 바꿔 해결하는 방법이다. 이런 문제를 MCMC와 같은 샘플링으로도 접근할 수 있으나 샘플링을 이용하는 경우 얻은 솔루션이 얼마나 $p$에 아까운지 알기 어렵고, 적절한 샘플링 기법을 선택하는 것 자체가 어렵기도 하다.
- Variational inference에서는 두 분포의 KL divergence를 이용한다. 참고로 이산 분포 $p$와 $q$의 KL divergence는 아래와 같이 정의된다.
 - 이 값은 $p$와 $q$에 무관하게 항상 0보다 크거나 같고, 만약 두 분포가 같다면 이 값은 0이 된다.
$$KL(q \Vert p) = \sum_x q(x) \log \frac{q(x)}{p(x)}$$

> - Variational inference에서는 미지의 분포 $p$를 아래와 같이 표현할 수 있다고 가정한다. ($Z(\theta)$는 partition function)
$$p(x_1, \cdots, x_n; \theta) = \frac{\tilde{p}(x_1, \cdots, x_n; \theta)}{Z(\theta)} = \frac{1}{Z(\theta)} \prod_k \phi_k(x_k;\theta)$$

> - $p$에 최대한 가까운 $q$를 찾기 위한 $KL\big( q(x) \Vert p(x) \big)$를 사용할 수 있고, 이를 $\tilde{p}$를 이용해 정리하면 아래와 같다.
$$\begin{align}
KL\big( q(x) \Vert p(x) \big) &= \sum_x q(x) \log \frac{q(x)}{p(x)} \\
&=\sum_x q(x) \log \big( q(x)\frac{Z(\theta)}{\tilde{p}(x)} \big)\\
&=\sum_x q(x) \bigg[ \log \frac{q(x)}{\tilde{p}(x)} + \log Z(\theta) \bigg] \\
&=KL \big(q(x) \Vert \tilde{p}(x) \big) + \log Z(\theta)
\end{align}$$

>  $$\log Z(\theta) = KL\big( q(x) \Vert p(x) \big) - KL \big(q(x) \Vert \tilde{p}(x) \big) \geq -J(\theta)$$

> - 이때 $KL\big( q(x) \Vert p(x) \big)$는 KL divergence의 특성(Jensen's inequality)에 의해 항상 0보다 크거가 같으므로 $- KL \big(q(x) \Vert \tilde{p}(x) \big)$는 $\log Z(\theta)$의 lower bound이다. 따라서 $- KL \big(q(x) \Vert \tilde{p}(x) \big)$를 최대화 하는 것은 $KL\big( q(x) \Vert p(x) \big)$를 최소화 하는 것과 같다.
- lower bound $-KL \big(q(x) \Vert \tilde{p}(x) \big)$를 $-J(q)$라고도 표기할 것이며, 이를 variational lower bound 혹은 evidence lower bound(ELBO)라 부른다.

> - 결국 $q(x)$를 찾기 위해서는 $-J(q)$를 최대화 혹은 $J(q)$를 최소화 해야하며, 아래와 같이 다시 정리할 수 있다.
$$\begin{align}
\log(Z(\theta)) &\geq -J(\theta) \\
&= -\sum_x q(x) \log \frac{q(x)}{\tilde{p}(x)} \\
&= -\mathbb{E}_{q(x)} \bigg[ \log q(x) - \log \tilde{p}(x)\bigg] \\
&= \mathbb{E}_{q(x)} \bigg[ \log \tilde{p}(x) - \log q(x)\bigg]
\end{align}$$

> - 그렇다면 어떤 형태의 $q$함수를 사용해야할까? 지수족 함수, 신경망, 가우시안 과정, 잠재변수 모형 등 다양한 선택지가 있다. 이들 중 아래와 같은 형태의 $q(x)$를 사용하는 방법을 'Mean-field inference'라 한다. (여기서 $q_i(x_i)$는 이산형 확률변수 $x_i$에 대한 확률분포이다.)
$$q(x) = q_1(x_1) q_2(x_2) \cdots q_n(x_n)$$
- Mean-field inferece는 아래 목적함수를 최소화 하는 식으로 풀게 된다. (이를 푸는 방법으로 $q_j$를 하나식 선택하고 나머지 $q_{\cdot}$는 고정한체로 $J(q)$를 최소화하는 $q_j$를 찾을 수 있는데, 이런 방법을 coordinate descent라 한다.)
$$\min_{q_1, \cdots, q_j, \cdots, q_n}J(q)$$

### Variational Auto-Encoder
아래 페이지의 'Auto-encoding variational Bayes'부터...
- https://ermongroup.github.io/cs228-notes/extras/vae/

> 데이터셋 $D = \{ x^1, x^2, \cdots, x^n \}$이 있을 때 아래와 같은 문제를 푼다고 해 보자.
1. 분포 $p$의 파라미터 $\theta$를 학습시키고 싶다.
2. $x$가 주어졌을 때 $z$에 대한 사후분포를 알고 싶다.(inference)
3. 부분적으로 유실된 이미지 $x$가 주어졌을 때 유실된 부분을 체우고 싶다.

> 이런 문제에는 아래와 같은 상황을 가정한다.
> - $p(z \mid x)$를 구하는 것이 쉽지 않다.
- 데이터셋 $D$가 굉장히 커서 한번에 처리하기 어렵다. subsample-batch를 사용해야한다.

> - (결론만 말하면) EM algorithm, mean fiend inference, sampling-based methods 등으로는 위 문제를 풀기 어렵다.

> - 위 문제를 푸는데 Auto-encoding variational bayes(AEVB)를 이용할 것이다. 앞서 다뤘다시피 AEVB는 variational inference에 기반하는데 이는 Evidence lower bound(ELBO)를 최대화하는 접근법이다. ELBO는 아래와 같다.
 - ? 앞서 다뤘던 수식과 차이가 있는데, 그 이유를 모르겠다.
$$\mathcal{L}(p_\theta, q_\theta) = \mathbb{E}_{q_{\theta}(z\mid x)} \bigg[ \log p_\theta(x, z) - \log q_\phi(z) \bigg]$$

> - Furthermore, instead of only doing inference, we will simultaneously perform learning via gradient descent on both ϕ and θ (jointly). Optimization over ϕ will keep ELBO tight around logp(x); optimization over θ will keep pushing the lower bound (and hence logp(x)) up. This is somewhat similar to how the EM algorithm optimizes a lower bound on the marginal likelihood.

> $$\begin{align}
KL \big(q(z \mid x) \Vert p(z \mid x) \big) &= \sum_z q(z \mid x) \log \frac{q(z \mid x)}{p(z \mid x)} \\
&= \sum_z q(z \mid x) \log \big( \frac{q(z \mid x)}{p(x, z)} p(x) \big)\\
&= \sum_z q(z \mid x) \big( \log \frac{q(z \mid x)}{p(x, z)} + \log p(x) \big)\\
&= KL\big( q(z \mid x) \Vert p(x, z) \big) + \log p(x) \\
\end{align}$$

> $$\begin{align}
\log p(x) &= KL \big(q(z \mid x) \Vert p(z \mid x) \big) - KL\big( q(z \mid x) \Vert p(x, z) \big)\\
&= KL \big(q(z \mid x) \Vert p(z \mid x) \big) + \mathbb{E}_{q_\theta (z \mid x)} \bigg[ \log p(x, z) - \log q(z \mid x)\bigg]
\end{align}$$

> - $x$는 evidence로 주어져 있으므로 $\log p(x)$는 상수이므로 $KL \big(q(z \mid x) \Vert p(z \mid x) \big)$를 최소화 하는 것은 $\mathbb{E}_{q_\theta (z \mid x)} \bigg[ \log p(x, z) - \log q(z \mid x)\bigg]$를 최대화 하는 것과 같다. 결국 evidence lower bound(ELBO)를 목적함수로 두고 이를 최대화 할 것이다. ($p$는 $\theta$로, $q$는 $\phi$로 parameterized되어 있다면 ELBO를 아래와 같이 표현할 수 있다.)
$$\mathcal{L}(p_\theta, q_\phi) = \mathbb{E}_{q_\phi (z \mid x)} \bigg[ \log p_\theta (x, z) - \log q_\phi(z \mid x)\bigg]$$
- 혹은 위 논의에서 $x$는 주어져 있으므로 $p(z \mid x)$에 대한 근사 함수로 $q(z \mid x)$ 대신 $q(z)$를 사용할수도 있다. 그 경우 ELBO는 아래와 같다.
$$\mathcal{L}(p_\theta, q_\phi) = \mathbb{E}_{q_\phi (z)} \bigg[ \log p_\theta (x, z) - \log q_\phi(z)\bigg]$$

> - ELBO를 아래와 같이 다시 표현할 수 있다.
$$\begin{align}
\mathcal{L}(p_\theta, q_\phi) &= - KL\big( q(z \mid x) \Vert p(x, z) \big) = \mathbb{E}_{q_\phi (z \mid x)} \bigg[ \log p_\theta (x, z) - \log q_\phi(z \mid x)\bigg] \\
&= \sum_z q_\phi (z \mid x) \log \frac{p_\theta (x, z)}{q_\phi (z \mid x)}\\
&= \sum_z q_\phi (z \mid x) \log \frac{p_\theta (x \mid z) p_\theta(z)}{q_\phi (z \mid x)}\\
&= \sum_z q_\phi (z \mid x) \bigg( \log p_\theta (x \mid z) + \log \frac{p_\theta(z)}{q_\phi (z \mid x)} \bigg)\\
&= \sum_z q_\phi (z \mid x) \bigg( \log p_\theta (x \mid z) + \log \frac{p_\theta(z)}{q_\phi (z \mid x)} \bigg)\\
&= \mathbb{E}_{q_\phi (z \mid x)} \bigg[ \log p_\theta (x \mid z) \bigg] - KL\big (q_\phi (z \mid x) \Vert p_\theta(z) \big)
\end{align}$$

> - 혹은 아래와 같다.
$$\begin{align}
\mathcal{L}(p_\theta, q_\phi) &= - KL\big( q(z) \Vert p(x, z) \big) = \mathbb{E}_{q_\phi (z)} \bigg[ \log p_\theta (x, z) - \log q_\phi(z)\bigg] \\
&= \sum_z q_\phi (z) \log \frac{p_\theta (x, z)}{q_\phi (z)}\\
&= \sum_z q_\phi (z) \log \frac{p_\theta (x \mid z) p_\theta(z)}{q_\phi (z)}\\
&= \sum_z q_\phi (z) \bigg( \log p_\theta (x \mid z) + \log \frac{p_\theta(z)}{q_\phi (z)} \bigg)\\
&= \sum_z q_\phi (z) \bigg( \log p_\theta (x \mid z) + \log \frac{p_\theta(z)}{q_\phi (z)} \bigg)\\
&= \mathbb{E}_{q_\phi (z)} \bigg[ \log p_\theta (x \mid z) \bigg] - KL\big (q_\phi (z) \Vert p_\theta(z) \big)
\end{align}$$

> - 위 식을 이용해 ELBO를 최대화하는 것의 의미를 다시 생각해보면 $\mathbb{E}_{q_\phi (z \mid x)} \bigg[ \log p_\theta (x \mid z) \bigg]$는 $z$라는 latent가 주어졌을 때 데이터 $x$를 관찰할 log-likelihood이고 이를 최대화 해야한다. 반면 $KL\big (q_\phi (z \mid x) \Vert p_\theta(z) \big)$는 $q(z)$와 $p(z)$의 차이이며 이를 최소화 해야한다.

> - encoder, decoder 관점에서 ELBO를 바라보면 $KL\big (q_\phi (z \mid x) \Vert p_\theta(z) \big)$를 최소화 하는 것은 encoder $q(z \mid x)$와 latent $p(z)$의 차이를 최소화 하는 것이다. 반면 $\mathbb{E}_{q_\phi (z \mid x)} \bigg[ \log p_\theta (x \mid z) \bigg]$을 최대화 하는 것은 입력 $x$와 decoder의 출력 $\hat{x}$와의 차이를 최소화 하는 역할을 한다.

> - $q_\phi(z \mid x)$는 encoder network이고 VAE에서는 $p(z)$를 $\mathcal{N}(0, 1)$로 두고, $KL\big (q_\phi (z \mid x) \Vert p_\theta(z) \big)$는 regularization term 역할을 하며 $q(z \mid x)$를 최대한 $\mathcal{N}(0, 1)$에 가깝게 만든다. 
- $p_\theta(x \mid z)$는 decoder network이고 $\mathbb{E}_{q_\phi (z \mid x)} \big[ \log p_\theta (x \mid z) \big]$를 reconstruction error라 한다.


###### The reparametrization trick

> - ELBO를 최대화 하려면 
$\mathcal{L}(p_\theta, q_\phi) = \mathbb{E}_{q_\phi (z \mid x)} \bigg[ \log p_\theta (x \mid z) \bigg] - KL\big (q_\phi (z \mid x) \Vert p_\theta(z) \big)$의 gradient($\phi$에 대한) $\nabla_{\phi, \theta} \mathcal{L}(p_\theta, q_\phi)$를 구해야 한다.
- $\nabla_{\phi, \theta} KL\big (q_\phi (z \mid x) \Vert p_\theta(z) \big)$는 $q_\phi (z \mid x)$가 gaussian일 경우 closed form으로 구할 수 있다. (원 논문 appendex B)
- 반면 $\nabla_\phi \mathbb{E}_{q_\phi (z \mid x)} \bigg[ \log p_\theta (x \mid z) \bigg]$는 구하기 어렵다. 
 - 이 계산을 쉽게 하기 위해 $z \sim q_{\mu, \sigma}(z) = \mathcal{N}(\mu, \sigma)$를 아래와 같이 변형한다.
 $$z = g_{\mu, \sigma}(\epsilon) = \mu + \epsilon \cdot \sigma, ~ ~ \epsilon \sim \mathcal{N}(0, 1)$$
 
> - $\nabla$ 계산과 파라미터가 명확하지 않다...

> - 이제 $\nabla_\phi \mathbb{E}_{z \sim q_\phi (z \mid x)} \bigg[ \log p_\theta (x \mid z) \bigg]$는 아래와 다시 표현할 수 있다. 이렇게하면 gradient가 expectation안으로 들어가므로 Monte Carlo 기법으로 이 값을 구할 수 있다.
$$\begin{align}
\nabla_\phi \mathbb{E}_{q_\phi (z \mid x)} \bigg[ \log p_\theta (x \mid z) \bigg] &= \nabla_\phi \mathbb{E}_{\epsilon \sim p(\epsilon)} \bigg[ \log p_\theta (x \mid g_{\mu, \sigma}(\epsilon)) \bigg] \\
&= \mathbb{E}_{\epsilon \sim p(\epsilon)} \bigg[ \nabla_\phi \log p_\theta (x \mid g_{\mu, \sigma}(\epsilon)) \bigg] 
\end{align}$$

###### All it together

> - Variational auto-encoder는 아래와 같이 특별한 형태의 $q$와 $p$를 사용한 세팅에 'Auto-encoding Variational Bayes'(AEVB)을 적용한 것이다.
 - $q(z \mid x) = \mathcal{N}(z;\vec{\mu}(x), \vec{\sigma}(x) \odot I)$
 - $p(x \mid z) = \mathcal{N}(x;\vec{\mu}(z), \vec{\sigma}(z) \odot I), ~ p(z) = \mathcal{N}(z;0, I)$
- 이런 세팅에서는 아래와 같은 ELBO를 얻을 수 있다.
$$\mathcal{L}(\theta, \phi; \mathbf{x}^{(i)}) \simeq \frac{1}{L} \sum_{l=1}^L \log p_\theta (\mathbf{x}^{(i)} \mid \mathbf{z}^{(i, l)} ) + \frac{1}{2} \sum_{j=1}^J \bigg( 1 + \log( (\sigma_j^{(i)})^2 - (\mu_j^{(i)})^2) - (\sigma_j^{(i)})^2 \bigg)$$

###### Implementation

- vae_tf_02

.

.

.

.

.